In [20]:
import pandas as pd
import numpy as np
import heapq
import os
from datetime import datetime, timedelta

# ==========================================
# 1. CONFIGURATION & CONSTANTS
# ==========================================

# INPUT FILE PATH (Update if needed)
INPUT_FILE = '/content/picklist_creation_data_for_hackathon_with_order_date.csv'

# [cite_start]Priority to Cutoff Time Mapping [cite: 122]
# Times < 12:00 are treated as the "Next Day" relative to the 9 PM start
CUTOFF_MAP = {
    'P1': '23:30', 'P2': '02:00', 'P3': '04:00',
    'P4': '06:00', 'P5': '07:00', 'P6': '09:00', 'P9': '11:00'
}

# [cite_start]Operational Time Costs (in minutes) [cite: 66, 67, 68, 69, 70]
TIME_PARAMS = {
    'setup': 2.0,        # Start to Zone (2 min)
    'bin_travel': 0.5,   # Intra-zone bin-to-bin (30 sec)
    'pick_unit': 5/60,   # Pickup time per unit (5 sec)
    'unload_order': 0.5, # Unloading at staging per order (30 sec)
    'staging': 2.0       # Zone to Staging (2 min)
}

# [cite_start]Shift Definitions [cite: 124]
SHIFTS = [
    {"name": "Night Shift 1", "start": "20:00", "end": "05:00", "count": 45},
    {"name": "Night Shift 2", "start": "21:00", "end": "07:00", "count": 35},
    {"name": "Morning Shift", "start": "08:00", "end": "17:00", "count": 40},
    {"name": "General Shift", "start": "10:00", "end": "19:00", "count": 30},
]

# [cite_start]Picking Operation Start Time [cite: 121]
START_HOUR_STR = "21:00"

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def get_cutoff_datetime(row_priority, base_date):
    """Calculates absolute cutoff datetime handling overnight transitions."""
    time_str = CUTOFF_MAP.get(row_priority, '11:00')
    h, m = map(int, time_str.split(':'))

    # If cutoff is AM (e.g., 02:00), it belongs to the next day relative to 9PM start
    days_offset = 1 if h < 12 else 0

    return base_date + timedelta(days=days_offset, hours=h, minutes=m)

def get_shift_timings(base_date):
    """
    Generates available picker slots.
    FIX: Correctly pushes shifts that end before 9PM to the NEXT day.
    """
    pickers = [] # Heap items: (available_time, picker_id, shift_end_time)
    p_id_counter = 0

    # [cite_start]Global Start Time: 9:00 PM on the base date [cite: 121]
    global_start = base_date + timedelta(hours=21)

    for shift in SHIFTS:
        s_h, s_m = map(int, shift['start'].split(':'))
        e_h, e_m = map(int, shift['end'].split(':'))

        # 1. Determine Initial Shift Start/End
        shift_start = base_date + timedelta(hours=s_h, minutes=s_m)

        # Handle overnight shifts crossing midnight (e.g. 20:00 -> 05:00)
        if e_h < s_h:
            shift_end = base_date + timedelta(days=1, hours=e_h, minutes=e_m)
        else:
            shift_end = base_date + timedelta(hours=e_h, minutes=e_m)

        # 2. FIX: Check if this shift happened in the "past" relative to 9 PM Start
        # Example: Morning Shift (08:00-17:00) ends before 21:00.
        # So we must schedule it for the NEXT morning.
        if shift_end <= global_start:
            shift_start += timedelta(days=1)
            shift_end += timedelta(days=1)

        # 3. Determine actual availability
        # Pickers start working at shift_start OR global_start, whichever is later.
        # Night shifts will be capped at global_start (21:00).
        # Morning shifts will start at shift_start (08:00 next day).
        actual_avail = max(shift_start, global_start)

        # Add pickers to pool
        for _ in range(shift['count']):
            heapq.heappush(pickers, (actual_avail, p_id_counter, shift_end))
            p_id_counter += 1

    return pickers

# ==========================================
# 3. CORE OPTIMIZATION ENGINE
# ==========================================

def optimize_warehouse():
    print(f"Loading data from {INPUT_FILE}...")

    # Optimizing memory by loading only necessary columns
    cols_to_load = [
        'dt', 'order_id', 'sku', 'order_qty', 'zone', 'bin_rank',
        'order_tag', 'store_id', 'pods_per_picklist_in_that_zone',
        'weight_in_grams', 'pod_priority'
    ]

    try:
        df = pd.read_csv(INPUT_FILE, usecols=cols_to_load)
    except FileNotFoundError:
        print("Error: File not found. Please check the path.")
        return

    # Pre-processing
    df['weight_kg'] = df['weight_in_grams'] / 1000.0
    base_date = pd.to_datetime(df['dt'].iloc[0])

    print("Calculating cutoff times...")
    df['cutoff_dt'] = df['pod_priority'].apply(lambda x: get_cutoff_datetime(x, base_date))

    # [cite_start]SORTING: Critical for Greedy Heuristic [cite: 95]
    # 1. Priority (Earliest Deadline) -> 2. Zone (Grouping) -> 3. Bin Rank (Travel Speed)
    print("Sorting dataset...")
    df = df.sort_values(by=['cutoff_dt', 'zone', 'bin_rank'])

    # Initialize Picker Heap with the FIX for 150 pickers
    available_pickers = get_shift_timings(base_date)
    print(f"Initialized {len(available_pickers)} pickers across all shifts (Target: 150).")

    # Metrics Setup
    summary_rows = []
    picklist_counter = 1
    total_skus_picked = 0

    # Track Whole Orders: map order_id -> set of picked SKUs
    order_progress = {}
    order_requirements = df.groupby('order_id')['sku'].nunique().to_dict()

    # Output Directory
    if not os.path.exists("picklists"):
        os.makedirs("picklists")

    # [cite_start]GROUP BY ZONE: A picklist must belong to ONE zone only [cite: 50]
    print("Starting optimization loop...")
    for zone, zone_group in df.groupby('zone', sort=False):
        records = zone_group.to_dict('records')
        idx = 0
        n = len(records)

        while idx < n:
            # If no pickers left (simulated end of all shifts), stop
            if not available_pickers:
                break

            # --- PICKLIST CREATION ---
            current_pl = []
            current_weight = 0.0
            current_units = 0
            current_stores = set()

            # Read first item to set constraints
            first_item = records[idx]
            max_pods = first_item['pods_per_picklist_in_that_zone']

            # [cite_start]Fragile Logic [cite: 64, 115]
            is_fragile_pl = str(first_item.get('order_tag', '')).lower() == 'fragile'
            weight_limit = 50.0 if is_fragile_pl else 200.0

            # Greedy Packing Loop
            while idx < n:
                item = records[idx]
                item_w = item['weight_kg'] * item['order_qty']

                # Check Constraints:
                # [cite_start]1. Units <= 2000 [cite: 52]
                if current_units + item['order_qty'] > 2000: break
                # [cite_start]2. Weight <= 200kg (or 50kg) [cite: 53]
                if current_weight + item_w > weight_limit: break
                # [cite_start]3. Pods Limit [cite: 116]
                if item['store_id'] not in current_stores and len(current_stores) >= max_pods: break
                # 4. Fragile Consistency Check (Optional: Strict separation)
                item_is_fragile = str(item.get('order_tag', '')).lower() == 'fragile'
                if is_fragile_pl != item_is_fragile: break

                # Add Item
                current_pl.append(item)
                current_weight += item_w
                current_units += item['order_qty']
                current_stores.add(item['store_id'])
                idx += 1

            if not current_pl:
                idx += 1
                continue

            # --- PICKER ASSIGNMENT ---
            # [cite_start]Get earliest available picker [cite: 12]
            avail_time, p_id, shift_end = heapq.heappop(available_pickers)

            # [cite_start]Calculate Duration [cite: 65-70]
            unique_bins = len(set(x['bin_rank'] for x in current_pl))
            unique_orders = len(current_stores)

            travel_time = (
                TIME_PARAMS['setup'] +
                (unique_bins * TIME_PARAMS['bin_travel']) +
                (current_units * TIME_PARAMS['pick_unit']) +
                (unique_orders * TIME_PARAMS['unload_order']) +
                TIME_PARAMS['staging']
            )

            finish_time = avail_time + timedelta(minutes=travel_time)

            # --- VALIDATION & RECORDING ---
            # [cite_start]Task must finish within shift [cite: 61]
            if finish_time <= shift_end:

                # [cite_start]Determine Type [cite: 110]
                if is_fragile_pl: pl_type = "fragile"
                elif len(set(x['sku'] for x in current_pl)) == 1: pl_type = "bulk"
                else: pl_type = "multi order"

                pl_filename = f"{current_pl[0]['dt']}_{picklist_counter:05d}"
                pl_valid_items = []

                # [cite_start]Check Cutoff: Late items = Wasted Effort (Zero Value) [cite: 56]
                for item in current_pl:
                    if finish_time <= item['cutoff_dt']:
                        total_skus_picked += item['order_qty']

                        # Update Whole Order Tracker
                        if item['order_id'] not in order_progress:
                            order_progress[item['order_id']] = set()
                        order_progress[item['order_id']].add(item['sku'])

                        pl_valid_items.append({
                            'SKU': item['sku'],
                            'Store': item['store_id'],
                            'Bin': '',
                            'Bin Rank': item['bin_rank']
                        })

                # Only save if useful items were picked
                if pl_valid_items:
                    # Save individual csv
                    pd.DataFrame(pl_valid_items).to_csv(f"picklists/{pl_filename}.csv", index=False)

                    # Log Summary
                    summary_rows.append({
                        'Picklist_date': current_pl[0]['dt'],
                        'picklist_no': picklist_counter,
                        'picklist_type': pl_type,
                        'stores_in_picklist': ",".join(map(str, current_stores))
                    })
                    picklist_counter += 1

                # Picker available for next task
                heapq.heappush(available_pickers, (finish_time, p_id, shift_end))
            else:
                # Shift ended; picker cannot take this task.
                # Task is dropped (Wasted Effort). Picker is removed from pool.
                pass

    # ==========================================
    # 4. FINAL RESULTS
    # ==========================================

    # [cite_start]Calculate Whole Orders [cite: 88]
    whole_orders_count = 0
    for oid, picked_set in order_progress.items():
        if len(picked_set) == order_requirements.get(oid, 0):
            whole_orders_count += 1

    print("\n" + "="*50)
    print("OPTIMIZATION RESULTS")
    print("="*50)
    print(f"Total SKU Units Picked (Primary): {total_skus_picked}")
    print(f"Total Picklists Generated: {len(summary_rows)}")
    print("="*50)

    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_csv("Summary.csv", index=False)
    print("Done. Generated 'Summary.csv' and 'picklists/' folder.")

if __name__ == "__main__":
    optimize_warehouse()

Loading data from /content/picklist_creation_data_for_hackathon_with_order_date.csv...
Calculating cutoff times...
Sorting dataset...
Initialized 150 pickers across all shifts (Target: 150).
Starting optimization loop...

OPTIMIZATION RESULTS
Total SKU Units Picked (Primary): 48906.0
Total Picklists Generated: 4693
Done. Generated 'Summary.csv' and 'picklists/' folder.


In [7]:
import pandas as pd
df_check_columns = pd.read_csv('picklist_creation_data_for_hackathon_with_order_date.csv')
print(df_check_columns.columns)

Index(['dt', 'order_date', 'location_code', 'order_id', 'sku', 'order_qty',
       'bin', 'zone', 'bin_rank', 'order_tag', 'store_id', 'floor', 'rack',
       'aisle', 'pods_per_picklist_in_that_zone', 'length_in_cm',
       'width_in_cm', 'height_in_cm', 'weight_in_grams', 'pod_priority'],
      dtype='object')


In [3]:
pip install ortools

# Task
Correct the `KeyError` by replacing `df['Weight']` with `df['weight_in_grams']` in the `solve_warehouse_optimization` function in cell `aLstjHaM7Z0q`, then re-execute the function and confirm the successful execution and the generated output.

## Correct Weight Column

### Subtask:
Modify the code in cell `aLstjHaM7Z0q` to use `df['weight_in_grams']` instead of `df['Weight']` when calculating `df['weight_kg']`.


**Reasoning**:
The subtask requires modifying the `solve_warehouse_optimization` function to correctly use the `weight_in_grams` column instead of the `Weight` column for calculating `weight_kg`. This fix will address the `KeyError` encountered in the previous execution.



In [13]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import heapq

# --- Configuration based on Problem Statement ---
START_TIME_SIM = "21:00" # Picking starts at 9PM [cite: 121]
CUTOFF_MAP = {
    'P1': '23:30', 'P2': '02:00', 'P3': '04:00',
    'P4': '06:00', 'P5': '07:00', 'P6': '09:00', 'P9': '11:00'
}

TIME_PARAMS = {
    'setup': 2.0,        # Start to Zone (min) [cite: 66]
    'bin_travel': 0.5,   # 30s bin-to-bin (min) [cite: 67]
    'pick_unit': 5/60,   # 5s per unit (min) [cite: 68]
    'unload_order': 0.5, # 30s per order (min) [cite: 69]
    'staging': 2.0       # Zone to Staging (min) [cite: 70]
}

def solve_warehouse_optimization(input_path):
    # 1. Scalable Data Loading & Processing
    df = pd.read_csv(input_path)
    df['weight_kg'] = df['weight_in_grams'] / 1000.0

    # Pre-calculate Cutoff Datetimes (Handling overnight wrap-around)
    base_date = pd.to_datetime(df['dt'].iloc[0])
    def get_cutoff_dt(priority):
        time_str = CUTOFF_MAP.get(priority, '11:00')
        hour = int(time_str.split(':')[0])
        # If hour < 12, it's the next day (since we start at 9 PM)
        days_offset = 1 if hour < 12 else 0
        return base_date + timedelta(days=days_offset, hours=hour, minutes=int(time_str.split(':')[1]))

    df['cutoff_dt'] = df['pod_priority'].apply(get_cutoff_dt)

    # Sort for Primary Objective: Cutoff Time [cite: 87]
    df = df.sort_values(by=['cutoff_dt', 'zone', 'bin_rank'])

    # 2. Initialize Picker Pool (Heap for throughput optimization) [cite: 124]
    pickers = [] # (next_available_time, picker_id, shift_end_time)
    p_id_gen = 0
    shifts = [
        ("Night 1", "20:00", "05:00", 45), ("Night 2", "21:00", "07:00", 35),
        ("Morning", "08:00", "17:00", 40), ("General", "10:00", "19:00", 30)
    ]
    for _, start, end, count in shifts:
        s_h = int(start.split(':')[0])
        e_h = int(end.split(':')[0])
        # Calculate start/end dates relative to simulation
        s_dt = base_date + timedelta(days=(1 if s_h < 12 else 0), hours=s_h)
        e_dt = base_date + timedelta(days=(1 if e_h < 12 else 0), hours=e_h)
        # Global start is 9 PM [cite: 121]
        actual_start = max(s_dt, base_date + timedelta(hours=21))
        for _ in range(count):
            heapq.heappush(pickers, (actual_start, p_id_gen, e_dt))
            p_id_gen += 1

    # 3. Execution & Metrics Tracking
    summary_rows = []
    successful_skus_count = 0
    order_fulfillment_tracker = {} # order_id: [total_skus, picked_skus]

    # Pre-fill tracker for whole order evaluation
    order_totals = df.groupby('order_id')['sku'].nunique().to_dict()
    for oid in order_totals:
        order_fulfillment_tracker[oid] = set()

    # Processing by Zone batches
    for zone, zone_group in df.groupby('zone'):
        items = zone_group.to_dict('records')
        idx = 0
        while idx < len(items) and pickers:
            # Capacity Constraints [cite: 52, 53, 64]
            max_pods = items[idx]['pods_per_picklist_in_that_zone']
            is_fragile = "fragile" in str(items[idx].get('order_tag', '')).lower()
            weight_limit = 50.0 if is_fragile else 200.0

            # Batch items into one picklist
            current_pl = []
            c_w, c_u, c_s = 0.0, 0, set()

            while idx < len(items):
                it = items[idx]
                it_w = it['weight_kg'] * it['order_qty']
                if (c_u + it['order_qty'] <= 2000 and c_w + it_w <= weight_limit and
                    (len(c_s) < max_pods or it['store_id'] in c_s)):
                    current_pl.append(it)
                    c_w += it_w
                    c_u += it['order_qty']
                    c_s.add(it['store_id'])
                    idx += 1
                else: break

            if not current_pl: continue

            # Picker Assignment Logic
            avail, p_id, p_end = heapq.heappop(pickers)
            num_bins = len(set(i['bin_rank'] for i in current_pl))
            duration = (TIME_PARAMS['setup'] + (num_bins * TIME_PARAMS['bin_travel']) +
                        (c_u * TIME_PARAMS['pick_unit']) + (len(c_s) * TIME_PARAMS['unload_order']) +
                        TIME_PARAMS['staging'])
            finish_time = avail + timedelta(minutes=duration)

            if finish_time <= p_end:
                # Evaluate if units contribute value [cite: 42, 55]
                for it in current_pl:
                    if finish_time <= it['cutoff_dt']:
                        successful_skus_count += it['order_qty']
                        order_fulfillment_tracker[it['order_id']].add(it['sku'])

                # Output Logging
                pl_type = "fragile" if is_fragile else ("bulk" if len(set(i['sku'] for i in current_pl)) == 1 else "multi order")
                summary_rows.append({
                    'Picklist_date': current_pl[0]['dt'],
                    'picklist_no': f"PL_{len(summary_rows)+1}",
                    'picklist_type': pl_type,
                    'stores_in_picklist': ",".join(map(str, c_s))
                })
                heapq.heappush(pickers, (finish_time, p_id, p_end))

    # 4. Final Metrics Calculation
    whole_orders_count = sum(1 for oid, picked in order_fulfillment_tracker.items()
                             if len(picked) == order_totals[oid])

    # Results Printing
    print("-" * 30)
    print(f"EVALUATION METRICS:")
    print(f"Total SKU Units picked before cutoff (Primary): {successful_skus_count}")
    print(f"Total Whole Orders completed before cutoff (Secondary): {whole_orders_count}")
    print("-" * 30)

    pd.DataFrame(summary_rows).to_csv("Summary.csv", index=False)
    return summary_rows

solve_warehouse_optimization("/content/picklist_creation_data_for_hackathon_with_order_date.csv")

------------------------------
EVALUATION METRICS:
Total SKU Units picked before cutoff (Primary): 48906.0
Total Whole Orders completed before cutoff (Secondary): 0
------------------------------


[{'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_1',
  'picklist_type': 'bulk',
  'stores_in_picklist': '398939.0,400263.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_2',
  'picklist_type': 'multi order',
  'stores_in_picklist': '394957.0,398693.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_3',
  'picklist_type': 'bulk',
  'stores_in_picklist': '400317.0,400263.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_4',
  'picklist_type': 'bulk',
  'stores_in_picklist': '395059.0,398693.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_5',
  'picklist_type': 'bulk',
  'stores_in_picklist': '395106.0,400263.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_6',
  'picklist_type': 'bulk',
  'stores_in_picklist': '394957.0,400357.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_7',
  'picklist_type': 'multi order',
  'stores_in_picklist': '395106.0,400317.0'},
 {'Picklist_date': '2025-12-08',
  'picklist_no': 'PL_8',
  'picklist

## Final Task

### Subtask:
Confirm the successful execution and the generated output, reporting the total SKU units and full orders completed before cutoff.


## Summary:

### Q&A
*   **Was the successful execution and the generated output confirmed, reporting the total SKU units and full orders completed before cutoff?**
    Yes, the `solve_warehouse_optimization` function executed successfully after the correction.
    A total of 48906.0 SKU units were picked before the cutoff time.
    0 whole orders were completed before the cutoff time.

### Data Analysis Key Findings
*   The `KeyError` encountered in the `solve_warehouse_optimization` function was successfully resolved by replacing the incorrect column reference `df['Weight']` with the correct column `df['weight_in_grams']` when calculating `df['weight_kg']`.
*   After the correction, the `solve_warehouse_optimization` function executed without errors.
*   The simulation reported that **48906.0** total SKU units were picked before their respective cutoff times.
*   Despite the high number of picked SKU units, **0** whole orders were completed before their cutoff times.

### Insights or Next Steps
*   Investigate why zero whole orders were completed despite a significant number of individual SKUs being picked. This could indicate issues with order segmentation, inventory availability for all items in an order, or ambitious cutoff times for multi-item orders.
*   Analyze the generated `Summary.csv` to gain more insights into the picklist types and stores included in each picklist, which might shed light on the order completion challenges.
